In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_store_scd 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH sap_assortment_site AS (
    SELECT * 
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_assortment_site 
    WHERE hash_id NOT IN (-1224047618379587442, -3290328116623744907)
)
,temp_1 AS (
    SELECT 
        * EXCEPT(date_from_new), 
        COALESCE(LEAD(date_from) OVER (PARTITION BY assortment, store_id ORDER BY date_to,last_update_date), DATE('9999-12-31')) AS date_from_to, 
        COALESCE(LEAD(date_to) OVER (PARTITION BY assortment, store_id ORDER BY date_to,last_update_date), DATE('9999-12-31')) AS date_to_to, 
        COALESCE(LAG(date_from) OVER (PARTITION BY assortment, store_id ORDER BY date_to,last_update_date), date_from) AS date_lag_from_to, 
        COALESCE(LAG(date_to) OVER (PARTITION BY assortment, store_id ORDER BY date_to,last_update_date), date_to) AS date_lag_to_to 
    FROM (
        SELECT 
            * EXCEPT(rn), 
            LEAD(date_from) OVER (PARTITION BY assortment, store_id ORDER BY date_to,last_update_date) AS date_from_new 
        FROM (
            SELECT 
                assortment,
                site AS store_id,	
                date_from,	
                date_to,
                is_delete,
                aedat AS last_update_date, 
                ROW_NUMBER() OVER (PARTITION BY assortment, site, lfdnr ORDER BY aedat DESC) AS rn
            FROM sap_assortment_site
            UNION ALL
            SELECT 
                a.asort AS assortment,
                a.locnr AS store_id,
                a.datab AS date_from,
                a.datbi AS date_to,
                0 is_delete,
                CAST(proc_date AS DATE) AS last_update_date,
                ROW_NUMBER() OVER (PARTITION BY a.locnr, a.asort ORDER BY reqtsn DESC, datapakid DESC, record DESC) AS rn 
            FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmdm40 a
        ) a 
        WHERE rn = 1 
    ) a 
    WHERE date_from_new > date_from OR date_from_new IS NULL
)
,temp_2 AS (
    SELECT 
        a.assortment,	
        a.store_id,	
        a.date_from, 
        a.last_update_date,
        CASE WHEN a.date_from_to < a.date_to THEN a.date_from_to ELSE a.date_to END AS date_to,
        a.is_delete
    FROM temp_1 a
)
,temp_3 AS (
    SELECT *, 
        LEAD(date_from) OVER (PARTITION BY assortment, store_id ORDER BY date_to, last_update_date) AS date_from_to, 
        LAG(date_to,1) OVER (PARTITION BY assortment, store_id ORDER BY date_to, last_update_date) AS date_lag_to_to, 
        LAG(date_from,1) OVER (PARTITION BY assortment, store_id ORDER BY date_to,last_update_date) AS date_lag_from
    FROM temp_2
)
,temp_4 AS (
    SELECT 
        a.assortment,	
        a.store_id,	
        date_from, 
        a.last_update_date,
        date_to,
        a.is_delete,
        CASE WHEN date_from = date_lag_to_to THEN NULL ELSE date_from END AS date_from_new,
        CASE WHEN date_to = date_from_to THEN NULL ELSE date_to END AS date_to_new
    FROM temp_3 a 
)
,temp_5 AS (
    SELECT 
        *, LAG(date_from,1) OVER (PARTITION BY assortment, store_id ORDER BY date_to,last_update_date) AS date_from_to 
    FROM temp_4 
    WHERE date_from_new IS NOT NULL OR date_to_new IS NOT NULL
)
SELECT
    a.assortment,	
    a.store_id,	
    COALESCE(a.date_from_new, a.date_from_to) AS start_date,
    a.date_to_new AS end_date,
    a.is_delete,
    0 block_by_site,
    CASE WHEN c.status = 'ACTIVE' THEN 0 ELSE 1 END AS block_by_assortment,
    CASE WHEN a.assortment NOT LIKE 'EX_%' THEN 0 ELSE 1 END AS block_by_ex,
    a.last_update_date
FROM temp_5 a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_site_active b 
        ON a.assortment = b.assortment AND a.store_id = b.store_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_desc c 
        ON a.assortment = c.assortment
WHERE a.date_to_new IS NOT NULL
""")